In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np
import math

In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.2989, 0.5870, 0.1140])

In [3]:
# convolution 2D
def conv2d(img, kernel, padding='same'):
    # Error handling
    if kernel.shape[0] != kernel.shape[1]:
        print("Use square filter!")
        return None
    
    if kernel.shape[0]%2 == 0 and kernel.shape[1]%2 == 0:
        print("filter size has to be odd")
        return None
    
    # Create result matrix
    result = np.zeros_like(img)
    
    # Calculate kernel size
    kernel_size = kernel.shape[0]
    
    # same padding
    if padding=='same':
        pad_size = int((kernel_size - 1) / 2)
    else:
        pad_size = 0
    img = np.pad(img, pad_size, mode='constant')
    
    # convolution
    width, height = img.shape
    offset = int(kernel_size / 2)
    for x in range(offset, width-offset):
        for y in range(offset, height-offset):
            result[x-offset, y-offset] = np.sum(img[x-offset:x+offset+1, y-offset:y+offset+1] * kernel)
            
    return result

In [4]:
def LoG(sigma=1, stencil=19):
    """
        Parameters:
            1. sigma : the width of the Gaussian kernel
            2. stencil : the overall image size of the filter (default 19x19)
            
        Return:
            1. a numpy array with the filter values
    """
    Log_filter = np.zeros((stencil,stencil))
    for x in range(stencil):
        for y in range(stencil):
            tmp = ((x - stencil/2)**2 + (y - stencil/2)**2)/(2*(sigma**2))
            Log_filter[x, y] = (-1/(math.pi*(sigma**4)))*(1-tmp)*np.exp(-tmp)
    return LoG_filter

In [5]:
def filterImg(img, fil):
    """
        Paramters:
            1. img : input image
            2. fil : filter
            
        Return:
            1. the filtered image
    """
    return conv2d(img.copy(), fil, padding="same")

In [6]:
# Log filters
sigmas = [sigma+1 for sigma in range(10)]
fig = plt.figure(figsize= (15,10))
for sigma in sigmas:
    LoG_filter = LoG(sigma=sigma)
    ax = fig.add_subplot(2, 5, sigma)
    ax.imshow(LoG_filter, cmap='gray')
    ax.set_title("sigma : %d\nstencil size : %dpixel" %(sigma, 19))

NameError: name 'LoG_filter' is not defined

<Figure size 1080x720 with 0 Axes>

In [ ]:
# read image
img = mpl.image.imread("sunflower.jpg")
img = rgb2gray(img)
plt.imshow(img, cmap='gray')

In [ ]:
# Log filters
sigmas = [sigma+1 for sigma in range(10)]
fig = plt.figure(figsize= (15,10))
for sigma in sigmas:
    LoG_filter = LoG(sigma=sigma)
    result = filterImg(img, LoG_filter)
    ax = fig.add_subplot(2, 5, sigma)
    ax.imshow(result, cmap='gray')
    ax.set_title("sigma : %d\nstencil size : %dpixel" %(sigma, 19))